this looks at the clusters from time perm cluster stats and gets their average length across electrodes, and also bins them in a histogram to see the distribution  
also should get the average cluster length

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


In [ ]:
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

group_mat = []

adjusted_sampling_freq = 1024

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror_0to1Test"
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Assuming `mat` is your array and `save_dir` is the directory where you want to save it
    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')

    # load the mat array
    mat = np.load(mat_save_path)

    group_mat.append(mat)

# Convert group_mat to a numpy array for calculation
group_mat_np = np.vstack(group_mat)  # Vertically stack the matrices

# Calculate the sum of each row for histogram
row_sums = group_mat_np.sum(axis=1)

row_sums_sig_electrodes = row_sums[row_sums > 0]

# Convert the sums to milliseconds
ms_per_sample = 1000 / adjusted_sampling_freq
row_sums_ms = row_sums * ms_per_sample
row_sums_sig_electrodes_ms = row_sums_sig_electrodes * ms_per_sample

# Assuming `row_sums_ms` contains your data
data = row_sums_ms

# Use numpy.histogram to get the bin edges
counts, bin_edges = np.histogram(data, bins='auto')

# Calculate bin widths as the difference between consecutive bin edges
bin_widths = np.diff(bin_edges)

# Print or analyze the bin widths
print("Bin widths:", bin_widths)

# Plotting the histogram
plt.hist(row_sums_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Electrodes in bin')
plt.show()

get onset times

In [ ]:
def find_clusters_and_onsets(data):
    onsets = []
    for row in data:
        in_cluster = False
        for idx, val in enumerate(row):
            if val > 0 and not in_cluster:
                onsets.append(idx)  # Save the onset time
                in_cluster = True  # Now we're inside a cluster
            elif val == 0:
                in_cluster = False  # Exiting the cluster
    return onsets

# Convert onsets to milliseconds
onsets = find_clusters_and_onsets(group_mat_np)
onsets_ms = [onset * ms_per_sample for onset in onsets]
onsets_s = [onset_ms / 1000 for onset_ms in onsets_ms]

fig, ax = plt.subplots()
ax.hist(onsets_s, bins='auto')
ax.set_title('Histogram of Cluster Onset Times')
ax.set_xlabel('Time from Stimulus Onset (s)')
ax.set_ylabel('Frequency (Hz)')

# Overlay a vertical line, adjust the time according to your context
# Here, you need to convert 0.5 seconds to milliseconds or the equivalent unit used in your `onsets_ms`
# Assuming 0.5 seconds should be directly comparable to the values in `onsets_ms`
ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)  # 0.5 sec = 500 ms

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()



In [ ]:
# Assuming `row_sums_ms` contains your data
data = onsets_ms

# Use numpy.histogram to get the bin edges
counts, bin_edges = np.histogram(data, bins='auto')

# Calculate bin widths as the difference between consecutive bin edges
bin_widths = np.diff(bin_edges)

# Print or analyze the bin widths
print("Bin widths:", bin_widths)

plot only sig electrodes

In [ ]:
# Assuming `row_sums_ms` contains your data
data = row_sums_sig_electrodes_ms

# Use numpy.histogram to get the bin edges
counts, bin_edges = np.histogram(data, bins='auto')

# Calculate bin widths as the difference between consecutive bin edges
bin_widths = np.diff(bin_edges)

# Print or analyze the bin widths
print("Bin widths:", bin_widths)
# Plotting the histogram
plt.hist(row_sums_sig_electrodes_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Sig Electrodes in bin')
plt.show()

get avg and std of row values, to see the avg and std of total sig time across electrodes

In [ ]:
avg_total_sig_time = row_sums_sig_electrodes_ms.mean()
std_total_sig_time = row_sums_sig_electrodes_ms.std()

print('avg total sig time: ', avg_total_sig_time)
print('stdev total sig time: ', std_total_sig_time)

now get the avg length and std of each cluster, to get the avg and std of cluster size across electrodes

In [ ]:
def find_clusters(row):
    """Find contiguous series of 1's and return their lengths."""
    return [len(cluster) for cluster in ''.join(map(str, row)).split('0') if cluster]

def compute_stats(group_mat_np):
    """Compute average and standard deviation of cluster lengths."""
    cluster_lengths = [find_clusters(row) for row in group_mat_np]
    avg_lengths = [np.mean(clusters) if clusters else 0 for clusters in cluster_lengths]
    std_lengths = [np.std(clusters) if clusters else 0 for clusters in cluster_lengths]
    
    # For significant electrodes only (excluding rows with all zeroes)
    sig_rows = group_mat_np[np.any(group_mat_np == 1, axis=1)]
    sig_cluster_lengths = [find_clusters(row) for row in sig_rows]
    sig_avg_lengths = [np.mean(clusters) if clusters else 0 for clusters in sig_cluster_lengths]
    sig_std_lengths = [np.std(clusters) if clusters else 0 for clusters in sig_cluster_lengths]
    
    return cluster_lengths, avg_lengths, std_lengths, sig_avg_lengths, sig_std_lengths

# Compute the statistics
cluster_lengths, avg_lengths, std_lengths, sig_avg_lengths, sig_std_lengths = compute_stats(group_mat_np) # honestly i only need sig_avg_lengths

# Convert sig_avg_lengths to a numpy array to perform element-wise multiplication
sig_avg_lengths_np = np.array(sig_avg_lengths)
sig_avg_lengths_ms = sig_avg_lengths_np * ms_per_sample

# Create histograms of average cluster lengths
# For all electrodes
# plt.hist(avg_lengths, bins=np.arange(min(avg_lengths), max(avg_lengths) + 1, 1), alpha=0.5, label='All Electrodes')
# For significant electrodes
plt.hist(sig_avg_lengths_ms, bins=np.arange(min(sig_avg_lengths_ms), max(sig_avg_lengths_ms) + 1, 1), alpha=0.5)
plt.title('Histogram of Average Cluster Lengths (ms)')
plt.xlabel('Average Cluster Length (ms)')
plt.ylabel('Number of Electrodes with this Avg Cluster Length')
plt.legend()
plt.show()

In [ ]:
min(sig_avg_lengths_ms)

In [ ]:
non_empty_clusters = [cluster for cluster in cluster_lengths if cluster]

In [ ]:
non_empty_clusters_sorted = sorted(non_empty_clusters, key=lambda x: min(x))

In [ ]:
non_empty_clusters_sorted_ms = [[length * ms_per_sample for length in cluster] for cluster in non_empty_clusters_sorted]
non_empty_clusters_sorted_ms